In [ ]:
%pip install pyvi transformers seqeval evaluate google-cloud-bigquery-storage protobuf

## Library

In [ ]:
import json
from tqdm import tqdm
from pyvi import ViTokenizer
import re
import itertools
import evaluate
import numpy as np
import pandas as pd
from collections import Counter
import torch.nn as nn
import torch
import os
from datasets import Dataset, DatasetDict 
from transformers import DataCollatorForTokenClassification
from transformers import AutoModel, RobertaTokenizerFast
from transformers import AutoModelForTokenClassification, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, DataCollatorWithPadding, EarlyStoppingCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from seqeval.metrics import classification_report, f1_score, accuracy_score

## Loading data

In [ ]:
print("Loading data files...")
TRAIN_PATH = '../data/04_model_input/train_dataset.json'
TEST_PATH = '../data/04_model_input/test_dataset.json'
DEV_PATH = '../data/04_model_input/dev_dataset.json'

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_json = json.load(f)

with open(TEST_PATH, 'r', encoding='utf-8') as f:
    test_json = json.load(f)

with open(DEV_PATH, 'r', encoding='utf-8') as f:
    dev_json = json.load(f)

print(f"-> Train raw docs: {len(train_json)}")
print(f"-> Test raw docs: {len(test_json)}")
print(f"-> Dev raw docs: {len(dev_json)}")

## Tokenizer Initialization

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(
    "vinai/phobert-base-v2",
    add_prefix_space=True
    )

## Prepare Data

In [ ]:
def tokenize_with_pyvi_offsets(text):
    tokenized_text = ViTokenizer.tokenize(text)
    raw_tokens = tokenized_text.split()
    
    tokens_map = []
    cursor = 0
    
    for token in raw_tokens:
        search_token = token.replace("_", " ")
        start = text.find(search_token, cursor)
        if start == -1:
            continue
            
        end = start + len(search_token)
        tokens_map.append({
            "text": token,
            "start": start,
            "end": end
        })
        
        cursor = end
        
    return tokens_map

In [ ]:
def convert_label_studio_to_ner_data(json_data):
    dataset = []
    
    for task in tqdm(json_data, desc="Converting with Pyvi"):
        if 'data' in task and 'text' in task['data']:
            raw_text = task['data']['text']
        else:
            continue

        tokens_map = tokenize_with_pyvi_offsets(raw_text)
        
        labels = ["O"] * len(tokens_map)

        if task.get('annotations'):
            results = task['annotations'][0]['result']
            for item in results:
                if item['type'] == 'labels':
                    label_name = item['value']['labels'][0]
                    start_char = item['value']['start']
                    end_char = item['value']['end']

                    for i, token in enumerate(tokens_map):
                        if token['start'] >= start_char and token['end'] <= end_char:
                            if token['start'] == start_char or (i > 0 and labels[i-1] == "O"):
                                labels[i] = f"B-{label_name}"
                            else:
                                labels[i] = f"I-{label_name}"
                        elif token['start'] < end_char and token['end'] > start_char:
                             if labels[i] == "O": # Ưu tiên gán nếu chưa có nhãn
                                labels[i] = f"B-{label_name}"
        
        sentence = [(t['text'], labels[i]) for i, t in enumerate(tokens_map)]
        if len(sentence) > 0:
            dataset.append(sentence)
            
    return dataset

In [ ]:
def ner_sentences_to_ner_format(sentences):
    data = {
        "tokens": [],
        "ner_tags": []
    }

    for sent in sentences:
        tokens = [tok for tok, lbl in sent]
        labels = [lbl for tok, lbl in sent]

        data["tokens"].append(tokens)
        data["ner_tags"].append(labels)

    return data

In [ ]:
from datasets import Dataset, DatasetDict

print("\nConverting to BIO format...")
train_sentences = convert_label_studio_to_ner_data(train_json)
dev_sentences   = convert_label_studio_to_ner_data(dev_json)
test_sentences  = convert_label_studio_to_ner_data(test_json)

train_data = ner_sentences_to_ner_format(train_sentences)
dev_data   = ner_sentences_to_ner_format(dev_sentences)
test_data  = ner_sentences_to_ner_format(test_sentences)

raw_datasets = DatasetDict({
    "train": Dataset.from_dict(train_data),
    "validation": Dataset.from_dict(dev_data),
    "test": Dataset.from_dict(test_data),
})

print(raw_datasets)

In [ ]:
train_sentences[0][:20]

In [ ]:
label_list = set()

for split in raw_datasets.keys():
    for seq in raw_datasets[split]['ner_tags']:
        for label in seq:
            label_list.add(label)

label_list = sorted(list(label_list)) 
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

print(f"Danh sách nhãn tìm thấy ({len(label_list)}): {label_list}")

In [ ]:
def tokenize_and_align_labels(example, label2id):
    tokenized = tokenizer(
        example["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=256,
        padding=False,
        return_attention_mask=True,
    )

    labels = []
    word_ids = tokenized.word_ids()

    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != prev_word_id:
            labels.append(label2id[example["ner_tags"][word_id]])
        else:
            tag = example["ner_tags"][word_id]
            if tag.startswith("B-"):
                tag = tag.replace("B-", "I-")
            labels.append(label2id[tag])
        prev_word_id = word_id

    tokenized["labels"] = labels
    return tokenized

In [ ]:
tokenized_datasets = raw_datasets.map(
    lambda x: tokenize_and_align_labels(x, label2id),
    batched=False,               
    remove_columns=raw_datasets["train"].column_names
)

print("Các cột trong dataset:", tokenized_datasets['train'].column_names)

print("\n--- KIỂM TRA MẪU TẬP TRAIN ---")
print(tokenized_datasets['train'][0]['labels'])

print("\n--- KIỂM TRA MẪU TẬP VALIDATION ---")
print(tokenized_datasets['validation'][0]['labels'])

In [ ]:
sample = tokenized_datasets['train']
print(len(sample["input_ids"]))
print(len(sample["labels"]))

## Train Phase

In [ ]:
def compute_class_weights(tokenized_train, num_labels, o_label_id=None):
    counts = Counter()
    for ex in tokenized_train:
        for l in ex["labels"]:
            if l == -100:
                continue
            counts[int(l)] += 1

    freqs = np.zeros(num_labels, dtype=np.float64)
    for i in range(num_labels):
        freqs[i] = counts.get(i, 0)

    freqs = np.clip(freqs, 1.0, None)

    weights = (freqs.sum() / freqs)
    weights = weights / weights.mean()

    if o_label_id is not None:
        weights[o_label_id] = min(weights[o_label_id], 0.5)

    return torch.tensor(weights, dtype=torch.float)

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        self._loss_fct = None
        self._loss_fct_device = None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
        logits = outputs.logits

        loss_device = logits.device
        weight = None
        if self.class_weights is not None:
            weight = self.class_weights.to(device=loss_device, dtype=torch.float)

        if (self._loss_fct is None) or (self._loss_fct_device != loss_device):
            self._loss_fct = nn.CrossEntropyLoss(weight=weight, ignore_index=-100)
            self._loss_fct_device = loss_device

        loss = self._loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    pad_to_multiple_of=8,
 )

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

print("Đang tải pre-trained PhoBERT...")
model = AutoModelForTokenClassification.from_pretrained(
    "vinai/phobert-base-v2",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
 )

model.resize_token_embeddings(len(tokenizer))
print(f"Đã resize model embedding về kích thước: {len(tokenizer)}")

from transformers import EarlyStoppingCallback, set_seed
set_seed(42)

o_label_id = label2id.get("O", None)
class_weights = compute_class_weights(tokenized_datasets["train"], len(label_list), o_label_id=o_label_id)
print("Class weights (sample):", class_weights[: min(10, len(class_weights))].tolist())
if o_label_id is not None:
    print("'O' label id:", o_label_id, "weight:", float(class_weights[o_label_id]))

training_args = TrainingArguments(
    output_dir="./phobert-ner-results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=50,
    weight_decay=0.01,
    max_grad_norm=1.0,
    warmup_ratio=0.06,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    label_smoothing_factor=0.0,  
    push_to_hub=False,
    report_to="none",
    fp16=True,
    logging_strategy="steps",
    logging_steps=50,
 )

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    class_weights=class_weights.to(model.device),
 )

In [ ]:
print("\n🚀 BẮT ĐẦU TRAINING...")
trainer.train()

print("\n✅ TRAINING HOÀN TẤT!")

save_path = "./phobert-ner-final"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model đã được lưu tại: {save_path}")

## Debug

In [17]:
def token_level_confusion(trainer, dataset, label_list):
    preds_out = trainer.predict(dataset)
    logits = preds_out.predictions
    labels = preds_out.label_ids
    preds = np.argmax(logits, axis=2)

    y_true = []
    y_pred = []
    for ps, ls in zip(preds, labels):
        for p, l in zip(ps, ls):
            if l == -100:
                continue
            y_true.append(int(l))
            y_pred.append(int(p))

    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(label_list))))
    df = pd.DataFrame(cm, index=label_list, columns=label_list)
    return df

## Test Phase

In [ ]:
MODEL_PATH = os.path.abspath("./phobert-ner-final")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
model.to(device)

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True
)

test_args = TrainingArguments(
    output_dir="./tmp",
    per_device_eval_batch_size=16,
    do_train=False,
    do_eval=False,
    do_predict=True,
    report_to="none"
)

trainer_ner = Trainer(
    model=model,
    args=test_args,
    data_collator=data_collator
)

In [ ]:
print("🚀 RUNNING TEST PHASE...")

test_predictions = trainer_ner.predict(tokenized_datasets["test"])

logits = test_predictions.predictions
labels = test_predictions.label_ids

preds = np.argmax(logits, axis=2)

In [ ]:
true_labels = []
true_preds  = []

for pred_seq, label_seq in zip(preds, labels):
    seq_labels = []
    seq_preds  = []

    for p, l in zip(pred_seq, label_seq):
        if l == -100:
            continue

        seq_labels.append(id2label[l])
        seq_preds.append(id2label[p])

    if len(seq_labels) > 0:
        true_labels.append(seq_labels)
        true_preds.append(seq_preds)

In [ ]:
print("===== TEST SET: ENTITY-LEVEL REPORT =====")
print(classification_report(true_labels, true_preds, digits=4))

print("Overall F1:", f1_score(true_labels, true_preds))
print("Overall Accuracy:", accuracy_score(true_labels, true_preds))

In [ ]:
def predict_from_testset(idx, raw_datasets, trainer, id2label):
    sample = tokenized_datasets["test"][idx]
    raw_tokens = raw_datasets["test"][idx]["tokens"]

    logits = trainer.model(
        input_ids=torch.tensor([sample["input_ids"]]).to(trainer.model.device),
        attention_mask=torch.tensor([sample["attention_mask"]]).to(trainer.model.device)
    ).logits

    preds = logits.argmax(dim=-1).squeeze().tolist()

    results = []
    for tok, pred, lab in zip(raw_tokens, preds, sample["labels"]):
        if lab == -100:
            continue
        results.append((tok, id2label[pred]))

    return results

In [ ]:
predict_from_testset(0, raw_datasets, trainer_ner, id2label)

In [ ]:
def predict_ner_table(idx, raw_datasets, tokenized_datasets, trainer, id2label):
    model = trainer.model
    model.eval()

    raw_tokens = raw_datasets["test"][idx]["tokens"]
    true_labels = raw_datasets["test"][idx]["ner_tags"]
    sample = tokenized_datasets["test"][idx]

    input_ids = torch.tensor([sample["input_ids"]]).to(model.device)
    attention_mask = torch.tensor([sample["attention_mask"]]).to(model.device)
    labels = sample["labels"]

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    preds = outputs.logits.argmax(dim=-1).squeeze().tolist()

    rows = []
    word_ptr = 0  

    for pred_id, label_id in zip(preds, labels):
        if label_id == -100:
            continue

        if word_ptr >= len(raw_tokens):
            break

        token = raw_tokens[word_ptr]
        true_label = true_labels[word_ptr]
        pred_label = id2label[pred_id]

        rows.append({
            "Token": token,
            "Thực tế (True)": true_label,
            "Dự đoán (Pred)": pred_label,
            "Kết quả": "✅" if true_label == pred_label else "❌"
        })

        word_ptr += 1

    df = pd.DataFrame(rows)

    correct = (df["Thực tế (True)"] == df["Dự đoán (Pred)"]).sum()

    print("=" * 60)
    print(" NER PREDICTION SAMPLE")
    print(f"Sentence #{idx+1}")
    print(f"-> Correct Predictions: {correct} / {len(df)} tokens")
    print("=" * 60)

    return df

In [ ]:
df = predict_ner_table(
    idx=0,
    raw_datasets=raw_datasets,
    tokenized_datasets=tokenized_datasets,
    trainer=trainer_ner,
    id2label=id2label
)

pd.set_option("display.max_rows", None)      
pd.set_option("display.max_columns", None)   
pd.set_option("display.width", None)        
pd.set_option("display.max_colwidth", None)

df